In [1]:
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import heapq
import folium



ModuleNotFoundError: No module named 'geopy'

In [29]:
lab_results = pd.read_csv("lab_results.csv")
lab_results.head(10)

,station_id,station_name,full_station_name,station_number,station_type,latitude,longitude,status,county_name,sample_code,sample_date,sample_depth,sample_depth_units,parameter,result,reporting_limit,units,method_name
0,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Conductance,3480,1.0,uS/cm,EPA 120.1
1,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Boron,7.7,0.1,mg/L,"Std Method 4500-B, C"
2,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Calcium,68,1.0,mg/L,EPA 215.2
3,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Chloride,758,0.1,mg/L,"Std Method 4500-Cl, B"
4,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Magnesium,59,0.1,mg/L,"Std Method 3500-Mg, E"
5,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Sodium,510,1.0,mg/L,"Std Method 3500-Na, D"
6,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Total Alkalinity,270,1.0,mg/L as CaCO3,EPA 310.1
7,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Total Hardness,412,1.0,mg/L as CaCO3,EPA 130.2
8,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,"Public, Review Status Unknown",Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,pH,8,0.1,pH Units,EPA 150.1
9,8166,01S04W04A001M,01S04W04A001M,01S04W04A001M,Groundwater,37.8788,-122.3027,"Public, Review Status Unknown",Alameda,WDIS_0744257,07/18/1975 09:50,NaN,Feet,Conductance,1430,1.0,uS/cm,EPA 120.1


In [36]:
santa_clara_lab_results = lab_results[lab_results["county_name"] == "Santa Clara"]
def haversine(coord1, coord2):
    return geodesic(coord1, coord2).meters
grouped_lat_long = santa_clara_lab_results.groupby(['latitude', 'longitude']).first().reset_index()
alviso_coords = (37.4262, -121.9761)
min_distance = float('inf')
closest_pairs = []
heapq.heapify(closest_pairs)

for index, row in grouped_lat_long.iterrows():
    coords_i = (row['latitude'], row['longitude'])
    distance = -1 * haversine(alviso_coords, coords_i)
    heapq.heappush(closest_pairs, (distance, coords_i))
    if len(closest_pairs) > 50:
        heapq.heappop(closest_pairs)

closest_points = [heapq.heappop(closest_pairs) for _ in range(len(closest_pairs))][::-1]

closest_df = pd.DataFrame(closest_points, columns=['distance', 'coordinates'])
closest_df["distance"] = closest_df["distance"] * -1
closest_df.head(10)

,distance,coordinates
0,676.311526,"(37.428, -121.9688)"
1,881.314163,"(37.4208, -121.9688)"
2,1035.858990,"(37.4171, -121.9735)"
3,1063.392622,"(37.4244, -121.9643)"
4,1204.214896,"(37.4208, -121.9643)"
5,1428.182661,"(37.4135, -121.9735)"
6,1452.940695,"(37.4171, -121.9643)"
7,1465.268394,"(37.428, -121.9597)"
8,1465.302813,"(37.4244, -121.9597)"
9,1522.431314,"(37.4135, -121.9826)"


In [37]:
stations_among_gaudalope = [(37.4208, -121.9688), (37.4135, -121.9643), (37.4063, -121.9552), (37.3991, -121.9414), (37.3955, -121.9414),
                             (37.3882, -121.9369), (37.3741, -121.933), (37.3702, -121.9277), (37.3672, -121.9247), (37.3411, -121.9025),
                             (37.3341, -121.9003), (37.3327, -121.8983), (37.3304, -121.8956), (37.3286, -121.8927), (37.3147, -121.8897),
                             (37.3015, -121.882), (37.2654, -121.8773), (37.263, -121.8755)]

outler_point_mom_wanted = [(37.4966, -122.1102)]

In [38]:
map_center = alviso_coords
mymap = folium.Map(location=map_center, zoom_start=12)

for _, row in closest_df.iterrows():
    folium.Marker(
        location=row['coordinates'],
        popup=f"Distance: {row['distance']} meters\nOther Column: {row['coordinates']}",
        icon=folium.Icon(color='blue')
    ).add_to(mymap)

# Save the map as an HTML file
mymap.save('closest_points_map.html')

additional_df = pd.DataFrame(stations_among_gaudalope + outler_point_mom_wanted, columns=['latitude', 'longitude'])
for _, row in additional_df.iterrows():
    folium.Marker(
        location=(row['latitude'], row['longitude']),
        popup="Additional Point",
        icon=folium.Icon(color='red')
    ).add_to(mymap)

# Save the map as an HTML file
mymap.save('closest_points_map.html')

In [43]:
santa_clara_lab_results['coordinates'] = list(zip(santa_clara_lab_results['latitude'], santa_clara_lab_results['longitude']))
total_stations = list(closest_df["coordinates"]) + stations_among_gaudalope + outler_point_mom_wanted
santa_clara_lab_results = santa_clara_lab_results[santa_clara_lab_results["coordinates"].isin(total_stations)]
santa_clara_lab_results.shape

(5600, 19)